In [46]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from datetime import datetime
import requests
import pandas as pd
import numpy as np

In [13]:
# currently handles only location=INDIA
# wasn't able to decode the srch variable .. but still works fine

def generate_glassdoor_url(job_name, page_number):
    BASE_URL = "https://www.glassdoor.co.in/Job/india-"
    job_name = job_name.replace(" ", "-").lower()
    srch = f"IL.0,5_IN115_KO{len('India-')},{len(job_name)+len('India-')}"
    ip = f"_IP{page_number}"
    return (f"{BASE_URL}{job_name}-jobs-SRCH_{srch}{ip}.htm?includeNoSalaryJobs=true")

In [14]:
# example
print(generate_glassdoor_url("Data Analyst", 1))

https://www.glassdoor.co.in/Job/india-data-analyst-jobs-SRCH_IL.0,5_IN115_KO6,18_IP1.htm?includeNoSalaryJobs=true


In [63]:
URL = generate_glassdoor_url("Data Analyst", 1)

In [64]:
def fetch_jobs(url):
    html_content = []
    
    options = Options()
    options.add_argument("window-size=1920,1080")
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    
    driver.get(url)
    driver.minimize_window()
    
#     Closes the signup prompt
#     try:
#         driver.find_element(By.XPATH,".//span[@class='SVGInline modal_closeIcon']").click()
#         time.sleep(2)
#     except NoSuchElementException:
#         time.sleep(2)
#         pass
    
    try:
        driver.find_element(By.XPATH,".//span[@class='SVGInline modal_closeIcon']").click()
        time.sleep(2)
    except NoSuchElementException:
        try:
            driver.find_element(By.XPATH, "//*[@id='JAModal']/div/div[2]/span/svg").click()
            time.sleep(2)
        except:
            time.sleep(2)
            pass
    
    done = False
    while not done:
        job_cards = driver.find_elements(By.XPATH,"//article[@id='MainCol']//ul/li[@data-adv-type='GENERAL']")
        for card in job_cards:
            card.click()
            time.sleep(1)

            #Closes the signup prompt
            try:
                driver.find_element(By.XPATH,".//span[@class='SVGInline modal_closeIcon']").click()
                time.sleep(2)
            except NoSuchElementException:
                time.sleep(2)
                pass

            html_content.append(driver.page_source)
            done = True
    
#     time.sleep(100)
    return html_content

In [65]:
html_content = fetch_jobs(URL)

/tmp/ipykernel_23948/1154738535.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [66]:
len(html_content)

30

In [23]:
html_content[0][:500]

'<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/" style="height: 100%;" lang="en"><head><script async="" src="https://sb.scorecardresearch.com/beacon.js"></script><script src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.en.yTSbWrSe458.O/m=auth2/exm=client/rt=j/sv=1/d=1/ed=1/rs=AHpOoo_5dpdVCMe_LPx1yH-hHA2M85TB-Q/cb=gapi.loaded_1?le=scs" async=""></script><script src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.en.yTSbWrSe458.O/m=cli'

In [68]:
URLS = [generate_glassdoor_url("Data Analyst", i) for i in range(1,31)]

In [70]:
print(datetime.now().strftime('%H:%M:%S'))

content=[]

for url in URLS:
    try:
        html_content = fetch_jobs(url)
        content.extend(html_content)
    except:
        print('Error')
    
print(datetime.now().strftime('%H:%M:%S'))

20:31:57


/tmp/ipykernel_23948/1154738535.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


21:25:52


In [71]:
len(content)

900

In [74]:
content[0]

'<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/" style="height: 100%;" lang="en"><head><script async="" src="https://sb.scorecardresearch.com/beacon.js"></script><script src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.en.yTSbWrSe458.O/m=auth2/exm=client/rt=j/sv=1/d=1/ed=1/rs=AHpOoo_5dpdVCMe_LPx1yH-hHA2M85TB-Q/cb=gapi.loaded_1?le=scs" async=""></script><script src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.en.yTSbWrSe458.O/m=client/rt=j/sv=1/d=1/ed=1/rs=AHpOoo_5dpdVCMe_LPx1yH-hHA2M85TB-Q/cb=gapi.loaded_0?le=scs" async=""></script><script type="text/javascript" async="" src="https://snap.licdn.com/li.lms-analytics/insight.min.js"></script><script src="https://connect.facebook.net/signals/config/1499075380345177?v=2.9.102&amp;r=stable" async=""></script><script async="" src="//connect.facebook.net/en_US/fbevents.js"></script><script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-NZVNV6T&amp;l=gdGlobals"></

In [75]:
# # Define a file name to save the results
# file_name = 'results_data-analyst.txt'

# # Open the file in write mode
# with open(file_name, 'w') as f:
#     # Write each result to a new line in the file
#     for result in content:
#         f.write(result + '\n' + '!@#$%^&' + '\n')
        
# # Print a message to confirm that the results have been saved
# print(f'Results have been saved to {file_name}')

Results have been saved to results_data-analyst.txt


In [136]:
# def fetch_jobs3(url):
#     html_content = []
    
#     options = Options()
#     options.add_argument("window-size=1920,1080")
#     driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    
#     driver.get(url)

#     time.sleep(1)
#     html_content = driver.page_source
#     soup = BeautifulSoup(html_content, 'html.parser')
#     a_tags = soup.find_all('a', {'class': 'jobLink css-1rd3saf eigr9kq2'})
#     print(len(a_tags))
    
#     component_links = [f"http://glassdoor.co.in{a_tags[i]['href']}" for i in range(len(a_tags))]
    
#     driver.quit()
    
#     return component_links


In [137]:
# temp = fetch_jobs3('https://www.glassdoor.co.in/Job/india-data-analyst-jobs-SRCH_IL.0,5_IN115_KO6,18_IP1.htm?includeNoSalaryJobs=true')

/tmp/ipykernel_23948/1511621327.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


30


In [139]:
# for i in temp:
#     print(i)

http://glassdoor.co.in/partner/jobListing.htm?pos=101&ao=1136043&s=58&guid=00000187c3f32c40b02604213e59bea2&src=GD_JOB_AD&t=SR&vt=w&cs=1_e0371af4&cb=1682619706725&jobListingId=1008611056856&jrtk=3-0-1gv1v6b3akugu801-1gv1v6b3ugfoj800-45469dc010f3d212-
http://glassdoor.co.in/partner/jobListing.htm?pos=102&ao=1136043&s=58&guid=00000187c3f32c40b02604213e59bea2&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_842b1aa3&cb=1682619706725&jobListingId=1008605657612&jrtk=3-0-1gv1v6b3akugu801-1gv1v6b3ugfoj800-6d404ad04b12d89d-
http://glassdoor.co.in/partner/jobListing.htm?pos=103&ao=1136043&s=58&guid=00000187c3f32c40b02604213e59bea2&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_e33c9dcb&cb=1682619706725&jobListingId=1008597318868&jrtk=3-0-1gv1v6b3akugu801-1gv1v6b3ugfoj800-c1b1e8e537005350-
http://glassdoor.co.in/partner/jobListing.htm?pos=104&ao=1136043&s=58&guid=00000187c3f32c40b02604213e59bea2&src=GD_JOB_AD&t=SR&vt=w&cs=1_9ff12d5f&cb=1682619706726&jobListingId=1008592480764&jrtk=3-0-1gv1v6b3akugu801-1gv1v6b3ugfoj800-2e70

# FINALL

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from datetime import datetime
import time
import requests
import pandas as pd
import numpy as np

In [2]:
def generate_glassdoor_url(job_name, page_number):
    base_url = "https://www.glassdoor.co.in/Job/india-"
    job_name = job_name.replace(" ", "-").lower()
    srch = f"IL.0,5_IN115_KO6,{len(job_name)+6}"
    ip = f"_IP{page_number}"
    return (f"{base_url}{job_name}-jobs-SRCH_{srch}{ip}.htm")

In [3]:
def fetch_jobs(url):
    html_content = []
    
    options = Options()
    options.add_argument("window-size=1920,1080")
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    
    driver.get(url)
    driver.minimize_window()
    
    try:
        driver.find_element(By.XPATH,".//span[@class='SVGInline modal_closeIcon']").click()
        time.sleep(2)
    except NoSuchElementException:
        try:
            driver.find_element(By.XPATH, "//*[@id='JAModal']/div/div[2]/span/svg").click()
            time.sleep(2)
        except:
            time.sleep(2)
            pass
    
    done = False
    while not done:
        job_cards = driver.find_elements(By.XPATH,"//article[@id='MainCol']//ul/li[@data-adv-type='GENERAL']")
        for card in job_cards:
            card.click()
            time.sleep(1)

            #Closes the signup prompt
            try:
                driver.find_element(By.XPATH,".//span[@class='SVGInline modal_closeIcon']").click()
                time.sleep(2)
            except NoSuchElementException:
                time.sleep(2)
                pass

            html_content.append(driver.page_source)
            done = True
    
#     time.sleep(100)
    return html_content

In [5]:
import re
from collections import namedtuple

LanguageRegex = re.compile(
    r"\b("
    r"ada|algol|assembly|awk|bash|basic|c(\+\+|#)?|cobol|coffeescript|dart|delphi|"
    r"elixir|elm|erlang|f#|fortran|go|groovy|haskell|html|java|javascript|julia|kotlin|"
    r"lisp|lua|matlab|objective-c|ocaml|pascal|perl|php|powershell|prolog|python|r|"
    r"ruby|rust|scala|scheme|scratch|shell|smalltalk|sql|swift|tcl|typescript|vb\.net|"
    r"visual basic"
    r")\b", re.IGNORECASE)

FrameworkRegex = re.compile(
    r"\b("
    r"angular|aurelia|backbone\.js|bootstrap|bulma|cakephp|cherrypy|django|docker|"
    r"ember\.js|express|fastapi|feathers\.js|flask|gatsby\.js|google cloud platform|"
    r"grails|hapi\.js|hugo|ibm cloud|ionic|jest|jquery|koa|laravel|loopback|meteor|"
    r"nestjs|next\.js|nuxt\.js|openstack|phoenix|polymer|pyramid|quasar|react\.js|"
    r"redux|restify|ruby on rails|sails\.js|sanic|serverless|sinatra|socket\.io|"
    r"strapi|styled-components|symfony|thunderbird|tornado|turboGears|uwsgi|vaadin|"
    r"vue\.js|vuex|wxWidgets|yarn|yesod|zepto"
    r")\b", re.IGNORECASE)

DatabaseRegex = re.compile(
    r"\b("
    r"cassandra|couchbase|couchdb|firebase|mongodb|ms sql|mysql|oracle|postgresql|"
    r"redis|sqlite"
    r")\b", re.IGNORECASE)

SkillsRegex = re.compile(
    r"\b("
    r"amazon web services|ansible|apache kafka|apache mesos|apache zookeeper|aws cloudformation|"
    r"aws cloudwatch|aws lambda|azure|chef|circleci|cloudflare|docker|gcp|git|github|"
    r"gitlab|jenkins|kubernetes|logstash|nginx|prometheus|puppet|redis|terraform|travis ci"
    r")\b", re.IGNORECASE)

In [6]:
def process_skills(skills):
    # Get the list of languages
    languages = [lang[0] for lang in skills['languages']]
    
    # Remove empty strings and duplicates, ignoring case
    languages = list(set([lang.strip().lower() for lang in languages if lang.strip()]))
    
    # Combine the results into a dictionary
    return {
        'languages': languages,
        'frameworks': list(set(skills['frameworks'])),
        'databases': list(set([db.strip().lower() for db in skills['databases'] if db.strip()])),
        'skills': list(set([skill.strip().lower() for skill in skills['skills'] if skill.strip()]))
}

In [7]:
import json
import re
from bs4 import BeautifulSoup

def parse_html_to_json(html,count):
    soup = BeautifulSoup(html, 'html.parser')

    company_name = ""
    job_title = ""
    location = ""
    job_description = ""
    avg_base_pay_est = ""
    company_rating = ""
    company_link = ""
    time_since_posted = ""
    company_skills = ""

    try:
        company_name = soup.find('div', {'class': 'css-87uc0g e1tk4kwz1'}).text
    except:
        company_name = "#N/A"
    try:
        job_title = soup.find('div', {'class': 'css-1vg6q84 e1tk4kwz4'}).text
    except:
        job_title = "#N/A"
    try:
        location = soup.find('div', {'class': 'css-56kyx5 e1tk4kwz5'}).text
    except:
        location = "#N/A"  
    try:
        avg_base_pay_est = soup.find('div', {'class': 'css-1bluz6i e2u4hf13'}).text
    except:
        avg_base_pay_est = "#N/A"  
    try:
        time_since_posted = soup.find('div', {'class': 'd-flex align-items-end pl-std css-1vfumx3'}).text
    except:
        time_since_posted = "#N/A"    
    try:
        company_rating = soup.find('span', {'class': 'css-1m5m32b e1tk4kwz2'}).text
    except:
        company_rating = "#N/A"

    try:   
        job_description = soup.find('div', {'class': 'css-jrwyhi e856ufb5'}).text
#         job_description = soup.text
    
        language_matches = LanguageRegex.findall(job_description)
        framework_matches = FrameworkRegex.findall(job_description)
        database_matches = DatabaseRegex.findall(job_description)
        skills_matches = SkillsRegex.findall(job_description)

        company_skills = {
            'languages': language_matches,
            'frameworks': framework_matches,
            'databases': database_matches,
            'skills': skills_matches
        }
        company_skills = process_skills(company_skills)
    except:
        job_description = "#N/A"
    a_tags = soup.find_all('a', {'class': 'jobLink css-1rd3saf eigr9kq2'}) 
    try:
        company_link="https://glassdoor.co.in"+a_tags[count]["href"]
    except:
        company_link="#N/A"    
        

    # create dictionary object from extracted data
    job_data = {
        'company_name': company_name,
        'job_title': job_title,
        'location': location,
        'job_description': job_description,
        'avg_base_pay_est': avg_base_pay_est,
        'company_rating': company_rating,
        'company_link': company_link,
        'time_since_posted': time_since_posted,
        'company_skills': company_skills
    }

    return json.dumps(job_data)


In [23]:
def scrape_job_results(JOB_NAME, NO_OF_PAGES):
    URLS = [generate_glassdoor_url(JOB_NAME, i) for i in range(1,NO_OF_PAGES+1)]

    json_list = []

    for url in URLS:
        html_content = fetch_jobs(url)
        for i in range(30):
            json_list.append(parse_html_to_json(html_content[i], i))
            
    return json_list

In [24]:
JOB_NAME = "ML Engineer"
NO_OF_PAGES = 2

temp = scrape_job_results(JOB_NAME, NO_OF_PAGES)

/tmp/ipykernel_162529/961610879.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [25]:
temp[0]

'{"company_name": "Planck Epoch", "job_title": "AI/ML Engineer", "location": "Calicut", "job_description": "We are looking for an expert in machine learning to help us extract value from our data. You will lead all the processes from data collection, cleaning, and preprocessing, to training models and deploying them to production.The ideal candidate will be passionate about artificial intelligence and stay up-to-date with the latest developments in the field.ResponsibilitiesUnderstanding business objectives and developing models that help to achieve them, along with metrics to track their progressManaging available resources such as hardware, data, and personnel so that deadlines are metAnalyzing the ML algorithms that could be used to solve a given problem and ranking them by their success probabilityExploring and visualizing data to gain an understanding of it, then identifying differences in data distribution that could affect performance when deploying the model in the real worldVe

In [26]:
len(temp)

60

In [28]:
# x = pd.DataFrame(temp)
# x

In [52]:
# ScrapeJobsList = ['Software Engineer', 'Data Scientist', 'Product Manager']

ScrapeJobsList = ['Software Engineer', 'Data Scientist', 'Product Manager', 'UX Designer',\
             'UI Designer', 'Web Developer', 'Mobile Developer', 'Backend Developer',\
             'Frontend Developer', 'Full-Stack Developer', 'DevOps Engineer',\
             'Cloud Solutions Architect', 'Machine Learning Engineer',\
             'Artificial Intelligence Specialist', 'Cybersecurity Analyst',\
             'Network Engineer', 'Database Administrator', 'IT Support Specialist',\
             'Technical Writer', 'Digital Marketing Manager']

In [53]:
def SCRAPE_ALL_JOB_RESULTS(ScrapeJobsList, NO_OF_PAGES):
    final_json = {}
    for job in ScrapeJobsList:
        final_json[job] = scrape_job_results(job, NO_OF_PAGES)
    return final_json

In [54]:
final_json = SCRAPE_ALL_JOB_RESULTS(ScrapeJobsList, 2)

/tmp/ipykernel_162529/961610879.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [55]:
len(final_json)

20

In [56]:
with open('trying_out_all_20_jobs_response.json', 'w') as f:
    json.dump(final_json, f)

In [37]:
# with open('final_jsoss.json', 'r') as f:
#     data = json.load(f)

In [34]:
# import json
# import pandas as pd

# def json_to_dataframe(json_str):
#     data = json.loads(json_str)

#     # Extract company skills data
#     languages = data['company_skills']['languages']
#     frameworks = data['company_skills']['frameworks']
#     databases = data['company_skills']['databases']
#     skills = data['company_skills']['skills']

#     # Create a dictionary with job details
#     job_dict = {
#         'company_name': data['company_name'],
#         'job_title': data['job_title'],
#         'location': data['location'],
#         'job_description': data['job_description'],
#         'avg_base_pay_est': data['avg_base_pay_est'],
#         'company_rating': data['company_rating'],
#         'time_since_posted': data['time_since_posted'],
#         'languages': ','.join(languages),
#         'frameworks': ','.join(frameworks),
#         'databases': ','.join(databases),
#         'skills': ','.join(skills)
#     }

#     # Create a DataFrame from the job details dictionary
#     df = pd.DataFrame.from_records([job_dict])
#     return df


In [35]:
# json_to_dataframe(parse_html_to_json(html_content[2]))

In [13]:
tech_jobs = ['Software Engineer', 'Data Scientist', 'Product Manager', 'UX Designer',\
             'UI Designer', 'Web Developer', 'Mobile Developer', 'Backend Developer',\
             'Frontend Developer', 'Full-Stack Developer', 'DevOps Engineer',\
             'Cloud Solutions Architect', 'Machine Learning Engineer',\
             'Artificial Intelligence Specialist', 'Cybersecurity Analyst',\
             'Network Engineer', 'Database Administrator', 'IT Support Specialist',\
             'Technical Writer', 'Digital Marketing Manager']

In [14]:
len(tech_jobs)

20